<!-- ---
title: ""
format: 
  revealjs:
    theme: custom.scss
    css: custom.css
    width: 1920
    height: 1080
    menu:
      side: right
      width: wide
    template-partials:
      - title-slide.html
    slide-number: c/t
    logo: "eclipse_logo_small.png"
    highlight-style: a11y
    incremental: false
    background-transition: fade
    footer: "©Philipp Pelz - FAU Erlangen-Nürnberg - Data Science for Electron Microscopy"
execute:
  eval: true
  echo: true
--- -->

# Minibatch Stochastic Gradient Descent

- Two extremes in gradient-based learning:
  - Full dataset (gradient descent)
  - Single examples (stochastic gradient descent)
- Each approach has drawbacks:
  - Gradient descent: Not data efficient for similar data
  - SGD: Not computationally efficient (poor vectorization)
- Minibatch SGD offers a middle ground

## Vectorization and Caches

### Hardware Considerations

- Multiple GPUs and servers require larger minibatches
  - 8 GPUs × 16 servers = minimum batch size of 128
- Single GPU/CPU considerations:
  - Multiple memory types (registers, L1/L2/L3 cache)
  - Different bandwidth constraints
  - Memory access patterns matter

### Performance Metrics

- Modern CPU capabilities:
  - 2GHz CPU with 16 cores and AVX-512
  - Can process up to 10¹² bytes/second
- GPU capabilities:
  - 100× better than CPU
- Memory bandwidth limitations:
  - Midrange server: ~100 GB/s
  - Memory access width: 64-384 bit

## Matrix Multiplication Strategies

### Different Approaches

1. Element-wise computation
2. Column-wise computation
3. Full matrix multiplication
4. Block-wise computation


---


### Performance Comparison


In [ ]:
#| label: matrix-mult
 
import d2l
import torch
import time
import numpy as np

class Timer:
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in a list."""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return the average time."""
        return sum(self.times) / len(self.times)

    def sum(self):
        """Return the sum of time."""
        return sum(self.times)

    def cumsum(self):
        """Return the accumulated time."""
        return torch.tensor(self.times).cumsum().tolist()

# Initialize matrices
A = torch.zeros(256, 256)
B = torch.randn(256, 256)
C = torch.randn(256, 256)
timer = Timer()

### Element-wise Computation


In [ ]:
#| label: element-wise
# Compute A = BC one element at a time
timer.start()
for i in range(256):
    for j in range(256):
        A[i, j] = torch.dot(B[i, :], C[:, j])
timer.stop()

### Column-wise Computation


In [ ]:
#| label: column-wise
# Compute A = BC one column at a time
timer.start()
for j in range(256):
    A[:, j] = torch.mv(B, C[:, j])
timer.stop()

---


### Full Matrix Multiplication


In [ ]:
#| label: full-matrix
# Compute A = BC in one go
timer.start()
A = torch.mm(B, C)
timer.stop()

gigaflops = [0.03 / i for i in timer.times]
print(f'performance in Gigaflops: element {gigaflops[0]:.3f}, '
      f'column {gigaflops[1]:.3f}, full {gigaflops[2]:.3f}')

## Minibatch Processing

### Why Use Minibatches?

- Computational efficiency
- Statistical properties:
  - Maintains gradient expectation
  - Reduces variance by factor of $b^{-\frac{1}{2}}$
  - $b$ = batch size

---


### Batch Size Trade-offs

- Too small:
  - Poor computational efficiency
  - High variance
- Too large:
  - Diminishing returns in variance reduction
  - Memory constraints
- Optimal: Balance between:
  - Computational efficiency
  - Statistical efficiency
  - Available memory

---


## Implementation

### Data Loading


In [ ]:
#| label: data-loading
#@save
d2l.DATA_HUB['airfoil'] = (d2l.DATA_URL + 'airfoil_self_noise.dat',
                           '76e5be1548fd8222e5074cf0faae75edff8cf93f')

#@save
def get_data_ch11(batch_size=10, n=1500):
    data = np.genfromtxt(d2l.download('airfoil'),
                         dtype=np.float32, delimiter='\t')
    data = torch.from_numpy((data - data.mean(axis=0)) / data.std(axis=0))
    data_iter = d2l.load_array((data[:n, :-1], data[:n, -1]),
                               batch_size, is_train=True)
    return data_iter, data.shape[1]-1

### Training Function


In [ ]:
#| label: training-fn
#@save
def sgd(params, states, hyperparams):
    for p in params:
        p.data.sub_(hyperparams['lr'] * p.grad)
        p.grad.data.zero_()

---


### Training Function


In [ ]:
#| label: training-fn1

def train_ch11(trainer_fn, states, hyperparams, data_iter,
               feature_dim, num_epochs=2):
    # Initialization
    w = torch.normal(mean=0.0, std=0.01, size=(feature_dim, 1),
                     requires_grad=True)
    b = torch.zeros((1), requires_grad=True)
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    # Train
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[0, num_epochs], ylim=[0.22, 0.35])
    n, timer = 0, d2l.Timer()
    for _ in range(num_epochs):
        for X, y in data_iter:
            l = loss(net(X), y).mean()
            l.backward()
            trainer_fn([w, b], states, hyperparams)
            n += X.shape[0]
            if n % 200 == 0:
                timer.stop()
                animator.add(n/X.shape[0]/len(data_iter),
                             (d2l.evaluate_loss(net, data_iter, loss),))
                timer.start()
    print(f'loss: {animator.Y[0][-1]:.3f}, {timer.sum()/num_epochs:.3f} sec/epoch')
    return timer.cumsum(), animator.Y[0]

## Performance Comparison

### Different Batch Sizes
::: {.columns}

::: {.column width="50%"}


In [ ]:
#| label: batch-comparison1
def train_sgd(lr, batch_size, num_epochs=2):
    data_iter, feature_dim = get_data_ch11(batch_size)
    return train_ch11(
        sgd, None, {'lr': lr}, data_iter, feature_dim, num_epochs)

# Compare different approaches
gd_res = train_sgd(1, 1500, 10)  # Full batch
sgd_res = train_sgd(0.005, 1)    # Single example
d2l.set_figsize([6, 3])
d2l.plot(*list(map(list, zip(gd_res, sgd_res))),
         'time (sec)', 'loss', xlim=[1e-2, 10],
         legend=['gd', 'sgd', 'batch size=100', 'batch size=10'])
d2l.plt.gca().set_xscale('log')

:::

::: {.column width="50%"}


In [ ]:
#| label: batch-comparison2
mini1_res = train_sgd(.4, 100)   # Medium batch
mini2_res = train_sgd(.05, 10)   # Small batch

# Plot results
d2l.set_figsize([6, 3])
d2l.plot(*list(map(list, zip( mini1_res, mini2_res))),
         'time (sec)', 'loss', xlim=[1e-2, 10],
         legend=['gd', 'sgd', 'batch size=100', 'batch size=10'])
d2l.plt.gca().set_xscale('log')

:::

:::
## Summary

- Vectorization benefits:
  - Reduced framework overhead
  - Better memory locality
  - Improved caching
- Minibatch SGD advantages:
  - Computational efficiency
  - Statistical efficiency
  - Memory efficiency
- Key considerations:
  - Batch size selection
  - Learning rate decay
  - Hardware constraints

## Exercises

1. Experiment with different batch sizes and learning rates
2. Implement learning rate decay
3. Compare with replacement sampling
4. Analyze behavior with duplicated data 